<a href="https://colab.research.google.com/github/ravikpasupula/GoogleColab.spark/blob/main/second_pyspark_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Spark program")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()


In [5]:
#import file into a dataframe
df = spark.read.csv('./sample_data/people-1000.csv', header=True)

In [24]:
from pyspark.sql.functions import col, asc, desc, lit, when
#print dataframe schema
df.printSchema()
#print top 5 records order by Index desc
df.sort(desc("Index")).show(5)

root
 |-- Index: string (nullable = true)
 |-- User Id: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Date of birth: string (nullable = true)
 |-- Job Title: string (nullable = true)

+-----+---------------+----------+---------+------+--------------------+--------------------+-------------+--------------------+
|Index|        User Id|First Name|Last Name|   Sex|               Email|               Phone|Date of birth|           Job Title|
+-----+---------------+----------+---------+------+--------------------+--------------------+-------------+--------------------+
|  999|Fb2FE369D1E171A|  Jermaine|   Phelps|  Male| wanda04@example.net|       (915)292-2254|   1971-08-31|    Ambulance person|
|  998|2adde51d8B8979E|     Cathy| Mckinney|Female|georgechan@exampl...|+1-750-774-4128x3...|   1918-05-13|Commercial/reside

In [12]:
#rename columns and load into a new df
df1 = df.select(col("Index").alias("Index_Id"), col("User Id").alias("User_Id"), col("First Name").alias("First_Name"), col("Sex").alias("Gender"), "Email", "Phone", col("Date of birth").alias("DOB"), col("Job Title").alias("Job_Title"))

df1.printSchema()

root
 |-- Index_Id: string (nullable = true)
 |-- User_Id: string (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Job_Title: string (nullable = true)



In [29]:
# Add new column "Constant" to existing dataframe df1 and give default value "C"
#the new column is added to result but not dataframe, in order to assign you need to assign df again
df1.withColumn('Constant', lit("C")).show(5)

+--------+---------------+----------+------+--------------------+------------------+----------+--------------------+--------+
|Index_Id|        User_Id|First_Name|Gender|               Email|             Phone|       DOB|           Job_Title|Constant|
+--------+---------------+----------+------+--------------------+------------------+----------+--------------------+--------+
|       1|8717bbf45cCDbEe|    Shelia|  Male| pwarner@example.org|      857.139.8239|2014-01-27|   Probation officer|       C|
|       2|3d5AD30A4cD38ed|        Jo|Female|fergusonkatherine...|   +1-950-759-8687|1931-07-26|              Dancer|       C|
|       3|810Ce0F276Badec|    Sheryl|Female| fhoward@example.org|     (599)782-0605|2013-11-25|                Copy|       C|
|       4|BF2a889C00f0cE1|   Whitney|  Male|zjohnston@example...|   +1-939-130-6258|2012-11-17|Counselling psych...|       C|
|       5|9afFEafAe1CBBB9|   Lindsey|Female|    elin@example.net|(390)417-1635x3010|1923-04-15| Biomedical engineer|  

In [34]:
#using case statement derive a new column whe
df2 = df1.withColumn('Constant', lit("C"))\
      .withColumn('IsUSA', when(df1.Phone.like("%+1-%"), 'Yes').otherwise('No'))

In [35]:
df2.printSchema()
df2.show(5)

root
 |-- Index_Id: string (nullable = true)
 |-- User_Id: string (nullable = true)
 |-- First_Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Job_Title: string (nullable = true)
 |-- Constant: string (nullable = false)
 |-- IsUSA: string (nullable = false)

+--------+---------------+----------+------+--------------------+------------------+----------+--------------------+--------+-----+
|Index_Id|        User_Id|First_Name|Gender|               Email|             Phone|       DOB|           Job_Title|Constant|IsUSA|
+--------+---------------+----------+------+--------------------+------------------+----------+--------------------+--------+-----+
|       1|8717bbf45cCDbEe|    Shelia|  Male| pwarner@example.org|      857.139.8239|2014-01-27|   Probation officer|       C|   No|
|       2|3d5AD30A4cD38ed|        Jo|Female|fergusonkatherine...|   +1-950-759